In [1]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter
import math
from scipy import stats

In [2]:
stocks = pd.read_csv('sp500.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2261348771213,
 'week52high': 152.05,
 'week52low': 69.6,
 'week52highSplitAdjustOnly': 146.27,
 'week52lowSplitAdjustOnly': 69.87,
 'week52change': 0.988859866139843,
 'sharesOutstanding': 17354475576,
 'float': 0,
 'avg10Volume': 90785968,
 'avg30Volume': 91481945,
 'day200MovingAvg': 128.1,
 'day50MovingAvg': 126.59,
 'employees': 147755,
 'ttmEPS': 3.83,
 'ttmDividendRate': 0.8418319530781752,
 'dividendYield': 0.006393787357769461,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-21',
 'nextEarningsDate': '2021-04-22',
 'peRatio': 36.37826721207324,
 'beta': 1.3412874978141784,
 'maxChangePercent': 51.70809339895707,
 'year5ChangePercent': 4.435342241960305,
 'year2ChangePercent': 1.7268089883039712,
 'year1ChangePercent': 1.037263676779998,
 'ytdChangePercent': 0.007746172692538487,
 'month6ChangePercent': 0.15321487737324974,
 'month3ChangePercent': -0.024138841772197616,
 'month1ChangePercent': 0.11717689660667369,
 'day30ChangePe